In [1]:
import re
import string
import operator
import functools
import glob
import os
import matplotlib.colors as mc
import nltk
import numpy as np
import textstat
import unicodedata
import pandas as pd

import textstat
from lexicalrichness import LexicalRichness
from collections import Counter

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag, map_tag
from nltk.tokenize import sent_tokenize

 # Feature Selection
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
# Cargar el archivo CSV en un DataFrame de pandas
df = pd.read_csv("intrinsic.csv")

In [4]:
df

,index,filename,text
0,0,suspicious-document00001.txt,The convening of this body gave rise to much d...
1,1,suspicious-document00002.txt,WOMAN\n\nMAN'S EQUAL.\n\nBY\n\nRev. THOS. WEBS...
2,2,suspicious-document00003.txt,"I do not remember that I was ""very angry"" abou..."
3,3,suspicious-document00004.txt,"Tulaji Angria was dark, inclined to be fat, ..."
4,4,suspicious-document00005.txt,Hiroshima was the primary target of the first ...
...,...,...,...
4747,4747,suspicious-document04748.txt,STORIES THAT\n\n WO...
4748,4748,suspicious-document04749.txt,"The legendary history of the place, both tempo..."
4749,4749,suspicious-document04750.txt,"The detective lunged toward her. ""Just what I ..."
4750,4750,suspicious-document04751.txt,Redactor's note: Italics are indicated by unde...


In [8]:
# Obtener los valores de la columna 'text'
textos = df["text"].values.astype('U')

# Crear una instancia de TfidfVectorizer
vectorizador = TfidfVectorizer(stop_words="english")

# Ajustar el vectorizador a los textos y transformar los textos en una matriz TF-IDF
matriz_tfidf = vectorizador.fit_transform(textos)

# Obtener las palabras únicas (características)
palabras = vectorizador.get_feature_names_out()

# Obtener las frecuencias TF-IDF de las palabras en todos los documentos
frecuencias_tfidf = np.array(matriz_tfidf.sum(axis=0)).flatten()

# Crear un DataFrame para almacenar las palabras y sus frecuencias TF-IDF
df_frecuencias = pd.DataFrame({"Palabra": palabras, "Frecuencia TF-IDF": frecuencias_tfidf})

# Ordenar el DataFrame por frecuencia TF-IDF en orden descendente
df_frecuencias_ordenadas = df_frecuencias.sort_values(by="Frecuencia TF-IDF", ascending=False)


# Obtener las 100 palabras más frecuentes y las 100 palabras menos frecuentes
palabras_mas_frecuentes = df_frecuencias_ordenadas.head(100)
palabras_menos_frecuentes = df_frecuencias_ordenadas.tail(100)

# Configurar pandas para mostrar el DataFrame completo de 100 filas
pd.set_option("display.max_rows", 100)

# Imprimir las palabras más frecuentes
print("Palabras más frecuentes:")
print(palabras_mas_frecuentes)

# Imprimir las palabras menos frecuentes
print("\nPalabras menos frecuentes:")
print(palabras_menos_frecuentes)

Palabras más frecuentes:
         Palabra  Frecuencia TF-IDF
245894      said         260.165765
174455       man         210.705990
284602      time         182.044632
167142    little         169.077946
166021      like         159.817780
80919        did         148.302556
122114     great         145.392545
180960       men         141.942684
120038      good         135.324976
75292        day         133.269047
201340       old         131.376479
165792      life         130.150161
189618        mr         127.345972
157194      know         121.935291
62189       come         113.864194
309377       way         113.525378
47152       came         113.059595
168433      long         108.061380
213275    people         100.711780
248928       say          98.663718
173577      make          96.622839
119264       god          93.842533
282528     think          92.715163
151567      just          92.707362
310442      went          92.243292
195402       new          91.609280
136

In [9]:
# Configurar pandas para mostrar el DataFrame completo de 100 filas
pd.set_option("display.max_rows", 100)

# Imprimir las palabras más frecuentes
print("Palabras más frecuentes:")
print(palabras_mas_frecuentes)

# Imprimir las palabras menos frecuentes
print("\nPalabras menos frecuentes:")
print(palabras_menos_frecuentes)

Palabras más frecuentes:
         Palabra  Frecuencia TF-IDF
245894      said         260.165765
174455       man         210.705990
284602      time         182.044632
167142    little         169.077946
166021      like         159.817780
80919        did         148.302556
122114     great         145.392545
180960       men         141.942684
120038      good         135.324976
75292        day         133.269047
201340       old         131.376479
165792      life         130.150161
189618        mr         127.345972
157194      know         121.935291
62189       come         113.864194
309377       way         113.525378
47152       came         113.059595
168433      long         108.061380
213275    people         100.711780
248928       say          98.663718
173577      make          96.622839
119264       god          93.842533
282528     think          92.715163
151567      just          92.707362
310442      went          92.243292
195402       new          91.609280
136

TF-IDF (Term Frequency-Inverse Document Frequency) es una medida utilizada en procesamiento de lenguaje natural para evaluar la importancia de una palabra en un documento dentro de una colección de documentos. Combina dos métricas: la frecuencia del término (TF) y la frecuencia inversa del documento (IDF).

La frecuencia del término (TF) mide la cantidad de veces que aparece un término específico en un documento. Cuanto más frecuente sea un término en un documento, mayor será su peso TF.

La frecuencia inversa del documento (IDF) mide la rareza de un término en la colección de documentos. Los términos que aparecen en pocos documentos tienen un IDF más alto y se consideran más importantes en la colección.

La fórmula general para calcular TF-IDF de un término en un documento es:

$TF-IDF = TF * IDF$

Donde TF se calcula dividiendo el número de veces que aparece el término en el documento por el número total de términos en el documento, y IDF se calcula como el logaritmo del cociente entre el número total de documentos en la colección y el número de documentos en los que aparece el término.

La idea detrás de TF-IDF es que las palabras que aparecen con frecuencia en un documento pero raramente en otros documentos tienen un valor TF-IDF más alto y son consideradas más importantes para ese documento en particular. Por el contrario, las palabras que aparecen en muchos documentos tienen un valor TF-IDF más bajo y son menos discriminativas.

El cálculo de TF-IDF se utiliza ampliamente en tareas como la recuperación de información, la clasificación de textos y la extracción de palabras clave, ya que permite identificar palabras clave y determinar la relevancia de los términos en un documento dentro de una colección más amplia.


El logaritmo se utiliza en IDF (Inverse Document Frequency o Frecuencia Inversa de Documento) para suavizar la escala del valor IDF y controlar el impacto de términos muy frecuentes o muy raros en un corpus de documentos.

El objetivo del IDF es penalizar los términos que aparecen en muchos documentos y, por lo tanto, tienen menos capacidad para discriminar y proporcionar información relevante. Al tomar el logaritmo del cociente entre el número total de documentos y el número de documentos que contienen un término específico, se reduce la escala del valor IDF.

Hay dos razones principales para utilizar el logaritmo en IDF:

Reducción de la escala: El logaritmo reduce la escala del valor IDF y lo convierte en una magnitud más manejable. Sin el logaritmo, los términos que aparecen en muy pocos documentos tendrían un valor IDF muy alto, mientras que los términos que aparecen en muchos documentos tendrían un valor IDF muy bajo. Esto podría generar un desequilibrio y una sobrevaloración de la importancia de los términos raros.

Igualación de la importancia relativa: Al aplicar el logaritmo, se reduce la diferencia entre los valores IDF de términos raros y términos comunes. Esto ayuda a igualar la importancia relativa de los términos y evita que unos pocos términos raros dominen la relevancia en la medida TF-IDF.

En resumen, el uso del logaritmo en IDF es una técnica para suavizar la escala y equilibrar la importancia relativa de los términos en el cálculo de TF-IDF. Ayuda a controlar el impacto de los términos más raros y a evitar una sobrevaloración o subvaloración extrema de la importancia de los términos en la medida TF-IDF.